In [14]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [35]:
navertoon = pd.read_csv("NaverToonInfo.csv")
navertoon.head(3)

,제목,평점,스토리작가,그림작가,업데이트 년도,형식,장르,이용가,요약,조회순
0,유미의 세포들,9.93,이동건,이동건,2020,에피소드,로맨스,12세 이용가,유미는 지금 무슨 생각을 하고 있을까?그녀의 머릿속에서 바쁘게 움직이는 세포들 이야기!,1
1,스피릿 핑거스,9.98,한경찰,한경찰,2018,스토리,로맨스,전체연령가,"평범한 여고생, 알록달록 이상한 그림모임에 들어가다!나만의 색을 찾아 떠나는 모험 ...",2
2,갓 오브 하이스쿨,9.64,박용제,박용제,2022,스토리,판타지,12세 이용가,"전국, 전세계 고등학생 중 가장 쎈 녀석을 뽑는 대회가 열린다.허구 100% 막장 ...",3


In [49]:
navertoon_df = navertoon[['제목','평점','스토리작가','그림작가','업데이트 년도','형식',
                          '장르','이용가','요약','조회순','장르형식']]

In [50]:
navertoon_df['장르형식'][0]
type(navertoon_df['장르형식'][0])

str

In [56]:
navertoon_df['장르형식']

0        에피소드,로맨스
1         스토리,로맨스
2         스토리,판타지
3         스토리,판타지
4         스토리,판타지
          ...    
1379    스토리,무협/사극
1380      스토리,드라마
1381      스토리,로맨스
1382       스토리,액션
1383      스토리,드라마
Name: 장르형식, Length: 1384, dtype: object

In [58]:
from sklearn.feature_extraction.text import CountVectorizer

navertoon_df['genres_literal'] = navertoon_df['장르형식'].apply(lambda x : ('').join(x))
navertoon_df['genres_literal'][0:3]

0    에피소드,로맨스
1     스토리,로맨스
2     스토리,판타지
Name: genres_literal, dtype: object

In [61]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2)) 

# 장르별 인덱스 사전
genre_mat = count_vect.fit_transform(navertoon_df['genres_literal'])
print(genre_mat.shape)

(1384, 43)


In [62]:
count_vect2 = CountVectorizer(min_df=1, ngram_range=(1, 1))  # min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2
genre_mat2 = count_vect2.fit_transform(navertoon_df['genres_literal'])
print(genre_mat2.shape)

(1384, 14)


In [63]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)

print(genre_sim.shape)
print(genre_sim[::-1])

(1384, 1384)
[[0.         0.33333333 0.33333333 ... 0.33333333 0.33333333 1.        ]
 [0.         0.33333333 0.33333333 ... 0.33333333 1.         0.33333333]
 [0.33333333 1.         0.33333333 ... 1.         0.33333333 0.33333333]
 ...
 [0.         0.33333333 1.         ... 0.33333333 0.33333333 0.33333333]
 [0.33333333 1.         0.33333333 ... 1.         0.33333333 0.33333333]
 [1.         0.33333333 0.         ... 0.33333333 0.         0.        ]]


In [64]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1] # [:,::-1] 2차원 배열의 역순정렬
genre_sim_sorted_ind[::-1]

array([[1383,  845, 1113, ...,  345,  346,    0],
       [ 760,  184, 1245, ..., 1072,  378,    0],
       [ 620,  181, 1187, ...,  577,  202,  351],
       ...,
       [ 691,  383, 1042, ...,  367,  372,    0],
       [ 620,  181, 1187, ...,  577,  202,  351],
       [   0,  510,  233, ...,  821,  820,  691]], dtype=int64)

In [83]:
title_toon = navertoon_df[navertoon_df['제목'] == '스피릿 핑거스']
title_toon

,제목,평점,스토리작가,그림작가,업데이트 년도,형식,장르,이용가,요약,조회순,장르형식,genres_literal,weighted_vote,가중평점
1,스피릿 핑거스,9.98,한경찰,한경찰,2018,스토리,로맨스,전체연령가,"평범한 여고생, 알록달록 이상한 그림모임에 들어가다!나만의 색을 찾아 떠나는 모험 ...",2,"스토리,로맨스","스토리,로맨스",9.706006,9.706006


In [84]:
# 갓파더의 인덱스를 변수로 저장하고,
title_index = title_toon.index.values

# 코사인 유사도가 비슷한 영화의 인덱스 10개 추출
similar_indexes = genre_sim_sorted_ind[title_index, :10]
similar_indexes = similar_indexes.reshape(-1)
    # 1차원 array로 변경
navertoon_df.iloc[similar_indexes].head(5)

,제목,평점,스토리작가,그림작가,업데이트 년도,형식,장르,이용가,요약,조회순,장르형식,genres_literal,weighted_vote,가중평점
620,미혼남녀의 효율적 만남,9.96,타리,타리,2022,스토리,로맨스,전체연령가,사랑만으론 부족하고 조건만으론 아쉬운 요즘 미혼남녀들의 효율추구 로맨스!마냥 현실적...,621,"스토리,로맨스","스토리,로맨스",9.814273,9.814273
181,걸어서 30분,9.97,이온도,이온도,2021,스토리,로맨스,전체연령가,"은하학원 위성은과 지구봉의 귀갓길 로맨스. 세상 가장 설레는 시간, 걸어서 30분",182,"스토리,로맨스","스토리,로맨스",9.752905,9.752905
1187,미친 악마,9.76,이에프,이에프,2022,스토리,로맨스,전체연령가,"평범한 대학생인 가린과 수한은 사실 전생에 원수지간이었다!?사밀의 환생 가린, 마한...",1188,"스토리,로맨스","스토리,로맨스",9.737508,9.737508
919,장단에 맞춰줘!,9.83,햇님,햇님,2021,스토리,로맨스,12세 이용가,초단신 보람과 일진농구부 최겸의너무 다른 반전 개그 로맨스!,920,"스토리,로맨스","스토리,로맨스",9.770849,9.770849
1190,오늘의 컨셉,9.71,일해,일해,2022,스토리,로맨스,전체연령가,남녀노소 취향 타지 않는 미인 곽사랑.빛나는 외모로 남들에게 고백만 받다가 처음으로...,1191,"스토리,로맨스","스토리,로맨스",9.708088,9.708088


In [85]:
def find_sim_toon_ver1(df, sorted_ind, title_name, top_n=10):
    # df 에서 title 컬럼값이 title_name 인 df를 추출
    title_toon = df[df['제목'] == title_name]
        
    # index 를 ndarray로 추출하고, 유사도 순으로 10개의 index 추출
    title_index = title_toon.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 데이터를 1차원으로 변환하고 해당 인덱스의 df 추출 
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

similar_toons = find_sim_toon_ver1(navertoon_df, genre_sim_sorted_ind, '스피릿 핑거스', 20)
similar_toons[['제목', '평점', '장르형식', '조회순']].head(5)

,제목,평점,장르형식,조회순
620,미혼남녀의 효율적 만남,9.96,"스토리,로맨스",621
181,걸어서 30분,9.97,"스토리,로맨스",182
1187,미친 악마,9.76,"스토리,로맨스",1188
919,장단에 맞춰줘!,9.83,"스토리,로맨스",920
1190,오늘의 컨셉,9.71,"스토리,로맨스",1191


In [86]:
navertoon_df[['제목','평점','조회순']].sort_values('평점', ascending=False)[:10]

,제목,평점,조회순
702,모퉁이 뜨개방,9.98,703
193,이상하고 아름다운,9.98,194
1097,율리,9.98,1098
547,먹이,9.98,548
538,홍차리브레,9.98,539
392,신비,9.98,393
167,유일무이 로맨스,9.98,168
523,고래별,9.98,524
522,구름의 이동속도,9.98,523
160,파도의 주인,9.98,161


In [77]:
#가중평점 반영

In [87]:
C = navertoon_df['평점'].mean()
m = navertoon_df['조회순'].quantile(0.6)

print('C:', round(C,3), 'm:',round(m,3))

C: 9.705 m: 830.8


In [88]:
def weighted_vote_average(record): 
    v = record['조회순']
    R = record['평점']
    
    return ((v/(v+m))*R)+((m/(m+v))*C)

navertoon_df['가중평점'] = navertoon_df.apply(weighted_vote_average, axis = 1)
navertoon_df.head(1)

,제목,평점,스토리작가,그림작가,업데이트 년도,형식,장르,이용가,요약,조회순,장르형식,genres_literal,weighted_vote,가중평점
0,유미의 세포들,9.93,이동건,이동건,2020,에피소드,로맨스,12세 이용가,유미는 지금 무슨 생각을 하고 있을까?그녀의 머릿속에서 바쁘게 움직이는 세포들 이야기!,1,"에피소드,로맨스","에피소드,로맨스",9.705617,9.705617


In [89]:
navertoon_df[['가중평점','제목','평점','조회순','장르형식']].sort_values('가중평점',ascending=False)[:10]

,가중평점,제목,평점,조회순,장르형식
1194,9.867362,언럭키 맨션,9.98,1195,"에피소드,드라마"
1304,9.867053,롭플롭,9.97,1305,"스토리,드라마"
1274,9.865587,냄새를 보는 소녀,9.97,1275,"스토리,드라마"
1253,9.864535,우리 사장님은 개,9.97,1254,"에피소드,일상"
1214,9.862524,Yes my boss(예스마이보스),9.97,1215,"에피소드,드라마"
1097,9.861698,율리,9.98,1098,"스토리,드라마"
1305,9.860989,물고기가 보이는 남자,9.96,1306,"스토리,드라마"
1270,9.859341,창공의 타이양사무소,9.96,1271,"에피소드,드라마"
1141,9.858547,굿 리스너,9.97,1142,"옴니버스,드라마"
1225,9.857138,영화원작] 모럴센스,9.96,1226,"스토리,로맨스"


In [90]:
navertoon_df1 = navertoon_df.drop(['weighted_vote'],axis=1)
navertoon_df1.head(1)

,제목,평점,스토리작가,그림작가,업데이트 년도,형식,장르,이용가,요약,조회순,장르형식,genres_literal,가중평점
0,유미의 세포들,9.93,이동건,이동건,2020,에피소드,로맨스,12세 이용가,유미는 지금 무슨 생각을 하고 있을까?그녀의 머릿속에서 바쁘게 움직이는 세포들 이야기!,1,"에피소드,로맨스","에피소드,로맨스",9.705617
